In [2]:
from typing import List
from functools import lru_cache

# Dynamic Programming
Use the solution to smaller subproblems to solve the problem. 

**State**
- what's changing at each step
- how does the state transition

**Top down DP**

    
    def dp(state):
        # check if state is in memo
        
        # Base case
        
        # solve dp(state) using dp(other states...)


## Knapsack




### 0-1 Knapsack

### Burst Ballons
You are given n balloons, indexed from 0 to n - 1. Each balloon is painted with a number on it represented by an array nums. You are asked to burst all the balloons.

If you burst the ith balloon, you will get nums[i - 1] * nums[i] * nums[i + 1] coins. If i - 1 or i + 1 goes out of bounds of the array, then treat it as if there is a balloon with a 1 painted on it.

Return the maximum coins you can collect by bursting the balloons wisely.

> State of nums changes each time we pop. Track it using dp(nums).

> We wanna maximize coins collected. So let dp(nums) return the maximum coins

> we can get in the current state of nums.

In [12]:
class PopBallons:
    def maxCoinsBruteForce(self, nums: List[int]) -> int:
        memo = {}   # {state of nums: max coins we can get from here}
        def dp(nums):
            # Check if nums is in memo.
            state = str(nums)
            if state in memo:
                return memo[state]
            
            # Base case
            if len(nums) == 3:
                return nums[1]
            
            # solve dp(nums) with the helps of other dp(...nums...)
            maxCoins = 0
            for i in range(1, len(nums)-1):
                # pop i
                coins = nums[i-1] * nums[i] * nums[i+1]
                popped = nums.pop(i)
                
                # after this pop, what are the "optimal" future popps gonna generate?
                futureCoins = dp(nums)
                maxCoins = max(maxCoins, coins + futureCoins)
                
                nums.insert(i, popped)
            
            memo[state] = maxCoins
            return maxCoins
            
            
        return dp([1] + nums + [1])
    
    def maxCoins(self, nums: List[int]) -> int:
        '''
        Once we pop ballon i, the rest of the ballons can be divided into left and right sections. Solve these sections separately and combine their future profits with the profit from ballon i. 
        
        However, above doesn't work. Since popping i affects i-1's right!!!
        
        Instead, think in reverse. Pop nums[left]...nums[i-1] and 
        nums[i+1]...nums[right] first (inclusive indices) and get their 
        futureCoins. 
        => Then pop nums[i] with new neighbor ballons nums[left-1] and nums[right+1]
        '''
        nums = [1] + nums + [1]
        
        @lru_cache
        def dp(left, right):

            # Base case
            if left > right:
                return 0
            
            # solve dp(nums) with the helps of other dp(...nums...)
            # [!] Pop nums[i] after popping nums[left:i] and nums[i+1:right+1]
            maxCoins = 0
            for i in range(left, right+1):
                # Pop ballons to the left & right of i first.
                futureCoins = dp(left, i-1) + dp(i+1, right)
                
                coins = nums[left-1] * nums[i] * nums[right+1]
                
                maxCoins = max(maxCoins, coins+futureCoins)
                
            
            return maxCoins
            
        return dp(1, len(nums)-2)
    
b = PopBallons()
b.maxCoins([8,2,6,8,9,8,1,4,1,5,3,0,7,7,0,4,2,2,5,5])

3830

### Decode Ways



In [4]:
class DecodeWays:
    def numDecodings(self, s: str) -> int:
        '''
        Take off 1 or 2 chars. Recursion on the rest of the string. 
        Each time we finish the whole string, count++, backtrack() and put the taken chars back.
        
        F(s[index:]) = (curr not '0') * F(s[index+1:]) + (s[index:index+2] is valid) *F(s[index+2:])
        '''
        # Invald: starts w/ 0
        if s[0] == '0':
            return 0
        
        memo = [None]*len(s)
        
        def backtrack(index):
            '''Number of ways to decode s[index:]'''
            # Base case
            if index >= len(s):
                return 1
            
            # Invalid case
            if s[index] == '0':
                return 0
            
            # MEMO!
            if memo[index]:
                return memo[index]
            
            # try decode the first one/two chars at startIndex, then
            # recursively carry out the decoding on the rest of the string.
            
            # take 1 off (guaranteed not 0)
            count = backtrack(index+1)

            # take 2 off, if decodable
            if (index+1 < len(s)) and (s[index], s[index+1]) <= ('2','6'):
                count += backtrack(index+2)

            memo[index] = count
            return count
        
        return backtrack(0)

# Linear DPs
Not so much like other DPs. 

### Buy and Sell Stocks 2



### Maximum Subarray

### Maximum Product Subarray
- 0's are separators. Restart `product` each time we hit a 0.
- The inputs are integers. The more nums we multiplied, the bigger the (absolute val) the result. 
- If there are even many negative integers, one pass is good.
- If there are odd any negative integers, 
    
        9, -1, -2, -3, 9
Must exclude exactly one of them. Do a pass from left->right. Another from right->left. 

In [3]:
class MaxProductSubarray:
    def maxProduct(self, nums: List[int]) -> int:
        '''
        
        9 -1 0 -2 -3 9
                     ^
        
        best = 9
        product = 6*9= 54
        '''
        best = float('-inf')     # best product
        product = 1   # current product
        
        for num in nums:
            product *= num
            
            if num == 0:
                best = max(best, 0)
                product = 1   # only for later iterations
                continue
            
            best = max(best, product)
            
        product = 1    
        for num in nums[-1::-1]:
            product *= num
            
            if num == 0:
                product = 1
                continue
            
            best = max(best, product)
        
        return best